In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests
import statsmodels.stats.multicomp as multicomp

In [2]:
data_df = pd.read_excel('../data/42255_2025_1311_MOESM4_ESM.xlsx', sheet_name='1H')

In [3]:
grp1, grp2, grp3 = data_df['HFD'].dropna().values, data_df['HFD+SAL'].dropna().values, data_df['HFD+SANA'].dropna().values

In [4]:
collect = [grp1, grp2, grp3]

In [5]:
len(np.concatenate([grp1, grp2, grp3]))

18

In [6]:
scipy.stats.f_oneway(grp1, grp2, grp3)

F_onewayResult(statistic=3.165867464704526, pvalue=0.07128335051187434)

In [8]:
comp = multicomp.MultiComparison(data=np.concatenate([grp1, grp2, grp3]), 
                                  groups=np.concatenate([['HFD']*len(grp1), 
                                                         ['HFD+SAL']*len(grp2), 
                                                         ['HFD+SANA']*len(grp3)]))
tbl, a1, a2 = comp.allpairtest(scipy.stats.ttest_ind, method= "bonf", alpha=0.05)
tbl

group1,group2,stat,pval,pval_corr,reject
HFD,HFD+SAL,-0.9206,0.3789,1.0,False
HFD,HFD+SANA,-3.0164,0.013,0.0389,True
HFD+SAL,HFD+SANA,-1.4401,0.1804,0.5412,False


In [9]:
scipy.stats.tukey_hsd(*[grp1, grp2, grp3]).pvalue

array([[1.        , 0.56385316, 0.05955134],
       [0.56385316, 1.        , 0.33526087],
       [0.05955134, 0.33526087, 1.        ]])

Here is Figure 1H:

![file](../img/pub_fig_1h.png)

This is described as involving a one-way ANOVA followed by Bonferroni post-hoc. Using the [source data](https://www.nature.com/articles/s42255-025-01311-z#Sec38) for this figure, we arrive at F(2, 16) = 3.166, p = 0.0713, which supports the authors' statement that treatment did not affect digested energy. However, the p-values for the post-hoc test appear to be wrong for Bonferroni post-hoc comparison (i.e., independent t-tests between groups and Bonferroni FWER correction on p-values for each pairwise test). The p-values shown here instead match the expected output of Tukey's post-hoc test, although with some inconsistencies in rounding.

| Comparison | Published p-value | Bonferroni p-value | Tukey p-value |
| -- | --- | --- | --- |
| HFD vs HFD+SANA | 0.06 | 0.0389 | 0.0596 |
| HFD vs HFD+SAL | 0.56 | 1.000 | 0.564 |
|  HFD+SAL vs HFD+SANA | 0.33 | 0.541 | 0.335 |

Could the authors clarify? 

The code for this analysis is available at [github.com/reeserich/cal_et_al_2025](https://github.com/reeserich/cal_et_al_2025).